### This is first try intent classification

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import collections
import json
import re
import numpy as np
import modeling
import tokenization
import tensorflow as tf

import pandas as pd

import tokenization

import sys

from extract_features import InputExample, InputFeatures, input_fn_builder, model_fn_builder

from extract_features import convert_examples_to_features, _truncate_seq_pair, read_examples

tf.logging.set_verbosity(tf.logging.ERROR)



In [2]:
BERT_BASE_DIR = '/Users/kolsha/Documents/Projects/Python/BERT/multi_cased_L-12_H-768_A-12'

init_checkpoint = BERT_BASE_DIR + '/bert_model.ckpt'

layer_indexes = [-1]

use_one_hot_embeddings = False

max_seq_length = 128

bert_config = modeling.BertConfig.from_json_file(BERT_BASE_DIR +'/bert_config.json')

tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_BASE_DIR+ '/vocab.txt', do_lower_case=False)

for (j, layer_index) in enumerate(layer_indexes):
    print(j, layer_index)

0 -1


In [3]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
  master=None,
  tpu_config=tf.contrib.tpu.TPUConfig(
      per_host_input_for_training=is_per_host)
)

In [4]:
def convert_lines_to_examples(lines):
    """Read a list of `InputExample`s from an input file."""
    examples = []
    unique_id = 0
    for line in lines:
        line = tokenization.convert_to_unicode(line)
        if not line:
            continue
        line = line.strip()
        text_a = None
        text_b = None
        m = re.match(r"^(.*) \|\|\| (.*)$", line)
        if m is None:
            text_a = line
        else:
            text_a = m.group(1)
            text_b = m.group(2)
        examples.append(
          InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
        unique_id += 1
    return examples

In [5]:
model_fn = model_fn_builder(
  bert_config=bert_config,
  init_checkpoint=init_checkpoint,
  layer_indexes=layer_indexes,
  use_tpu=False,
  use_one_hot_embeddings=use_one_hot_embeddings)

estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  config=run_config,
  predict_batch_size=32)
    
def get_embeddings(lines):
    result = []
    
    examples = convert_lines_to_examples(lines)
    
    features = convert_examples_to_features(
      examples=examples, seq_length=max_seq_length, tokenizer=tokenizer)
    
    unique_id_to_feature = {}
    for feature in features:
        unique_id_to_feature[feature.unique_id] = feature
    


    input_fn = input_fn_builder(
      features=features, seq_length=max_seq_length)
    
    pred = estimator.predict(input_fn, yield_single_examples=True)
    
    for p in pred:
        #print(p['layer_output_0'].shape)
        unique_id = int(p["unique_id"])
        feature = unique_id_to_feature[unique_id]
        #print(feature.tokens)
        layer_output = p["layer_output_0"]
        r = np.array([round(float(x), 6) for x in layer_output[0:1].flat])
        result.append(r)
    
    return np.array(result)

import random

def predict_input(lines, estimator, lb_enc):
    hz_answers = [
        'Переформулируй пожалуйста',
        'Это точно по русски было?',
        'Я Вам не смогу помочь',
    ]
    embeddings = get_embeddings(lines)
    pred = estimator.predict_proba(embeddings)
    result = []
    for p in pred:
        a_max = np.argmax(p, axis=None)
        if p[a_max] > 0.65:
            result.append(lb_enc.inverse_transform([a_max])[0])
        else:
            result.append(random.choice(hz_answers))
            
        
    return result#lb_enc.inverse_transform(pred)

In [6]:
%%time
get_embeddings(['Тест на ', 'Это был тест на ']).shape

CPU times: user 7.49 s, sys: 956 ms, total: 8.45 s
Wall time: 6.56 s


(2, 768)

# Tests

In [6]:
data = pd.read_csv('intents.csv')
data.head()

,label,text
0,Как заблокировать сим-карту,Заблокировать эту сим-карту
1,Как заблокировать сим-карту,"Здравствуйте, хочу заблокировать эту симкарту..."
2,Как заблокировать сим-карту,Можно заблокировать симкарту?
3,Как заблокировать сим-карту,Здравствуйте. Хочу заблокировать симкарту
4,Как заблокировать сим-карту,как заблокировать симкарту


In [8]:
# data['text'] = data['text'].str.replace("[^a-zA-Zа-яА-Я]", " ")
data.head()

,label,text
0,Как заблокировать сим-карту,Заблокировать эту сим-карту
1,Как заблокировать сим-карту,"Здравствуйте, хочу заблокировать эту симкарту..."
2,Как заблокировать сим-карту,Можно заблокировать симкарту?
3,Как заблокировать сим-карту,Здравствуйте. Хочу заблокировать симкарту
4,Как заблокировать сим-карту,как заблокировать симкарту


In [9]:
%%time
embeddings = get_embeddings(data['text'].values)
print(embeddings.shape)

(877, 768)
CPU times: user 18min 50s, sys: 17.4 s, total: 19min 7s
Wall time: 6min 24s


In [10]:
np.save('embeddings_old', embeddings)

In [9]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

In [10]:
embeddings = np.load('embeddings_old.npy')
le = preprocessing.LabelEncoder()
le.fit(data['label'].values)
list(le.classes_)

['Верните деньги',
 'Как заблокировать сим-карту',
 'Как заказать обратный звонок?',
 'Как поменять домашний регион?',
 'Как узнать кодовое слово?',
 'Не приходят смс',
 'Переведите на человека',
 'Позвоните мне!',
 'Почему вы подключаете услуги без моего согласия?',
 'Что за  тарифный план?']

In [11]:
Y = le.transform(data['label'].values)

In [12]:
print(Y)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 

In [13]:
cv = StratifiedShuffleSplit(n_splits=10, random_state=42) #, shuffle=True


In [27]:
forest = RandomForestClassifier(
    criterion='entropy', n_estimators=140,
#     max_features=None,
#     max_depth=17,
#     min_samples_leaf=2,
    random_state=42)

In [ ]:
f1_mac_best = 0
f1_mic_best = 0
f1_mac_all = []
for (train, test) in cv.split(embeddings, Y):
    forest.fit(embeddings[train], Y[train])
    y_pred = forest.predict(embeddings[test])
    
    f1_mac = f1_score(Y[test], y_pred, average='macro')
    f1_mic = f1_score(Y[test], y_pred, average='micro')
    
    f1_mac_all.append(f1_mac)
    print("F1 Macro: {}".format(f1_mac) )
    print("F1 Micro: {}".format(f1_mic) )
    
#     break
    
    if f1_mac > f1_mac_best:
#         best_logreg = copy.copy(logreg)
        f1_mac_best = f1_mac
        f1_mic_best = f1_mic
    
print("BEST F1 Macro: {}".format(f1_mac_best) )
print("BEST F1 Micro: {}".format(f1_mic_best) )

f1_mac_all = np.array(f1_mac_all)
f1_mac_avg = f1_mac_all.mean()
print("AVG  F1 Macro: {}".format(f1_mac_avg) )
print(f1_mac_all.std())

BEST F1 Macro: 0.7565208890443331 <br/>
BEST F1 Micro: 0.7840909090909092

# xgboost

In [29]:
import xgboost as xgb

In [ ]:
f1_mac_best = 0
f1_mic_best = 0
f1_mac_all = []
for (train, test) in cv.split(embeddings, Y):
#     forest.fit(embeddings[train], Y[train])
#     y_pred = forest.predict(embeddings[test])
    
    xg_train = xgb.DMatrix(embeddings[train], label=Y[train])
    xg_test = xgb.DMatrix(embeddings[test], label=Y[train])
    # setup parameters for xgboost
    param = {
            'n_estimators': 300,
            'max_depth': 6,
            'objective': 'multi:softmax',
            'learning_rate': 0.05,
            'subsample': 0.9,
            'colsample_bytree': 0.9
            #'eta': 0.1, 
            }

    param['silent'] = 1
    param['nthread'] = 4
    param['num_class'] = len(le.classes_)

    watchlist = [(xg_train, 'train')]#, (xg_test, 'test')
    num_round = 300
    bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=30)
    # get prediction
    y_pred = bst.predict(xg_test)
    f1_mac = f1_score(Y[test], y_pred, average='macro')
    f1_mic = f1_score(Y[test], y_pred, average='micro')
    
    f1_mac_all.append(f1_mac)
    print("F1 Macro: {}".format(f1_mac) )
    print("F1 Micro: {}".format(f1_mic) )
    
#     break
    
    if f1_mac > f1_mac_best:
#         best_logreg = copy.copy(logreg)
        f1_mac_best = f1_mac
        f1_mic_best = f1_mic
    
print("BEST F1 Macro: {}".format(f1_mac_best) )
print("BEST F1 Micro: {}".format(f1_mic_best) )

f1_mac_all = np.array(f1_mac_all)
f1_mac_avg = f1_mac_all.mean()
print("AVG  F1 Macro: {}".format(f1_mac_avg) )
print(f1_mac_all.std())

BEST F1 Macro: 0.819804920434346 <br/>
BEST F1 Micro: 0.8522727272727273


In [96]:
from sklearn.model_selection import GridSearchCV

In [102]:

params_grid_xgb = {
          'nthread':[4],
          'objective': ['multi:softmax'],
          'reg_alpha': [0, 0.5],
          'reg_lambda': [0, 0.5],
          'gamma': [0, 0.5],
          'subsample': [0.7, 1],
          'colsample_bytree':[0.7, 1],
          'max_depth': [1, 9, 20],
          'learning_rate': [0.05],
          'n_estimators': [100, 500]
}

xgb_model = xgb.XGBClassifier()


clf = GridSearchCV(xgb_model, params_grid_xgb, n_jobs=-1, 
                   cv=cv, 
#                    scoring='roc_auc',
                   verbose=True, refit=True)
print(params_grid_xgb)

{'nthread': [4], 'objective': ['multi:softmax'], 'reg_alpha': [0, 0.5], 'reg_lambda': [0, 0.5], 'gamma': [0, 0.5], 'subsample': [0.7, 1], 'colsample_bytree': [0.7, 1], 'max_depth': [1, 9, 20], 'learning_rate': [0.05], 'n_estimators': [100, 500]}


In [ ]:
clf.fit(embeddings, Y)
print("Fit end")
print(clf.best_params_, clf.best_score_)

# LogisticRegression

In [14]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=42, solver='lbfgs',#'lbfgs', 'newton-cg'
                            multi_class='multinomial',
                            max_iter=3000,
#                             C=0.75
                           )
import copy

In [15]:
f1_mac_best = 0
f1_mic_best = 0
best_logreg = None
f1_mac_all = []
for (train, test) in cv.split(embeddings, Y):
    logreg.fit(embeddings[train], Y[train])
    y_pred = logreg.predict(embeddings[test])
    
    f1_mac = f1_score(Y[test], y_pred, average='macro')
    f1_mic = f1_score(Y[test], y_pred, average='micro')
    
    f1_mac_all.append(f1_mac)
    print("F1 Macro: {}".format(f1_mac) )
    print("F1 Micro: {}".format(f1_mic) )
    
#     break
    
    if f1_mac > f1_mac_best:
        best_logreg = copy.copy(logreg)
        f1_mac_best = f1_mac
        f1_mic_best = f1_mic
    
print("BEST F1 Macro: {}".format(f1_mac_best) )
print("BEST F1 Micro: {}".format(f1_mic_best) )

f1_mac_all = np.array(f1_mac_all)
f1_mac_avg = f1_mac_all.mean()
print("AVG  F1 Macro: {}".format(f1_mac_avg) )
print(f1_mac_all.std())

F1 Macro: 0.8641095860661079
F1 Micro: 0.8863636363636365
F1 Macro: 0.9253339253339254
F1 Micro: 0.9545454545454546
F1 Macro: 0.8997229622229622
F1 Micro: 0.9090909090909091
F1 Macro: 0.9300216352847932
F1 Micro: 0.9431818181818182
F1 Macro: 0.8594143982379276
F1 Micro: 0.9090909090909091
F1 Macro: 0.9699559699559699
F1 Micro: 0.9772727272727273
F1 Macro: 0.9457973069473771
F1 Micro: 0.9545454545454546
F1 Macro: 0.9146915823386412
F1 Micro: 0.9204545454545455
F1 Macro: 0.8813650516436895
F1 Micro: 0.9318181818181818
F1 Macro: 0.9179254923820142
F1 Micro: 0.9431818181818182
BEST F1 Macro: 0.9699559699559699
BEST F1 Micro: 0.9772727272727273
AVG  F1 Macro: 0.9108337910413409
0.0334371708320174


In [ ]:
BEST F1 Macro: 0.9699559699559699
BEST F1 Micro: 0.9772727272727273
AVG  F1 Macro: 0.9108290028012584
0.03344453642061686

In [57]:
samples = [
    'Здравствуйте, меня зовут Павел, я не понимаю, как можно заблокировать симкарту?',
    'СОС, ХЕЛП, нужно срочно блокнуть симку, СРОЧНО',
    'Алоха, как восстановить симкарту?', # bad sample
    
    'Это снова Павел, где мои деньги, негодяи?',
    'Ребят, случайно отправил деньги не туда',
    
    'А ты точно человек, а скажи чтонибудь по человечьи?'
]
print(predict_input(samples, best_logreg, le))

['Я Вам не смогу помочь', 'Как заблокировать сим-карту', 'Я Вам не смогу помочь', 'Вы кто такие?', 'Денег не будет', 'Переведите на человека']


In [16]:
from sklearn.externals import joblib

In [17]:
joblib.dump(best_logreg, "best_logreg_3_2_19", compress=9)

['best_logreg_3_2_19']

In [186]:
emb = get_embeddings(samples)
pred = best_logreg.predict_proba(emb)

In [192]:
# # np.set_printoptions(precision=1)
# for p in pred:
#     a_max = np.argmax(p, axis=None)
    
# #     ind = np.unravel_index(np.argmax(a, axis=None), a.shape)

1
1
3
0
0
6


In [18]:
VK_API_ACCESS_TOKEN = 'aada2900ca3b69e310739de830ea662e44cf4d159bbeffc7259edc21382a28d811b89cf52dca486642830'  # Ключ доступа сообщества (для получения адреса сервера)
GROUP_ID = 177447412          # ID группы, должны быть в ней администратором
VK_API_VERSION = '5.74'	  # Используемая версия VK API

In [21]:
import vk
from requests import *
import os

import re

session = vk.Session(access_token = VK_API_ACCESS_TOKEN)
api = vk.API(session, v = VK_API_VERSION)

# Первый запрос к LongPoll: получаем server и key
longPoll = api.groups.getLongPollServer(group_id = GROUP_ID)

server, key, ts = longPoll['server'], longPoll['key'], longPoll['ts']
while True:
    # Последующие запросы: меняется только ts
    longPoll = post('%s'%server, data = {'act': 'a_check',
                                         'key': key,
                                         'ts': ts,
                                         'wait': 1,
                                         'version': 2}).json()
    ts = longPoll['ts']
    if longPoll['updates'] and len(longPoll['updates']) != 0:
        for update in longPoll['updates']:
            if update['type'] == 'message_new':
                print('message_new')
                # Помечаем сообщение от этого пользователя как прочитанное
                api.messages.setActivity(peer_id = update['object']['user_id'], type='typing', group_id = GROUP_ID)
                api.messages.markAsRead(peer_id = update['object']['user_id'])
                text = update['object']['body']
                text = text.replace("[^a-zA-Zа-яА-Я]", " ")
#                 text = re.sub("\s\s+" , " ", text)
                name = api.users.get(user_ids = update['object']['user_id'])[0]['first_name']
                res  = predict_input([text], best_logreg, le)
                api.messages.send(peer_id = update['object']['user_id'],
                                  message = '{}'.format(res[0]))
                break


    # Меняем ts для следующего запроса
    


message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new
message_new


KeyboardInterrupt: 

# SVC

In [20]:
from sklearn.svm import SVC
import copy

In [36]:
svc_clf = SVC(probability=True,
              random_state=42,
              gamma='auto',
              kernel='linear'
              
             )

In [37]:
f1_mac_best = 0
f1_mic_best = 0
best_svc_clf = None
f1_mac_all = []
for (train, test) in cv.split(embeddings, Y):
    svc_clf.fit(embeddings[train], Y[train])
    y_pred = svc_clf.predict(embeddings[test])
    
    f1_mac = f1_score(Y[test], y_pred, average='macro')
    f1_mic = f1_score(Y[test], y_pred, average='micro')
    
    f1_mac_all.append(f1_mac)
    print("F1 Macro: {}".format(f1_mac) )
    print("F1 Micro: {}".format(f1_mic) )
    
#     break
    
    if f1_mac > f1_mac_best:
        best_svc_clf = copy.copy(svc_clf)
        f1_mac_best = f1_mac
        f1_mic_best = f1_mic
    
print("BEST F1 Macro: {}".format(f1_mac_best) )
print("BEST F1 Micro: {}".format(f1_mic_best) )

f1_mac_all = np.array(f1_mac_all)
f1_mac_avg = f1_mac_all.mean()
print("AVG  F1 Macro: {}".format(f1_mac_avg) )
print(f1_mac_all.std())

F1 Macro: 0.8765841485406702
F1 Micro: 0.8977272727272727
F1 Macro: 0.9093203093203093
F1 Micro: 0.9431818181818182
F1 Macro: 0.9496195847289042
F1 Micro: 0.9431818181818182
F1 Macro: 0.9249417249417249
F1 Micro: 0.9431818181818182
F1 Macro: 0.8460469319609103
F1 Micro: 0.8977272727272727
F1 Macro: 0.8895904891785898
F1 Micro: 0.9204545454545455
F1 Macro: 0.9202764827764827
F1 Micro: 0.9318181818181818
F1 Macro: 0.9302096052096052
F1 Micro: 0.9318181818181818
F1 Macro: 0.9016662349995683
F1 Micro: 0.9545454545454546
F1 Macro: 0.8723997069368737
F1 Micro: 0.9090909090909091
BEST F1 Macro: 0.9496195847289042
BEST F1 Micro: 0.9431818181818182
AVG  F1 Macro: 0.9020655218593638
0.029638318567245418
